<a href="https://colab.research.google.com/github/rpolea/CE888/blob/main/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [111]:
df= pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/jester-data-1.csv',header=None)

In [112]:
df.head(10)
df.shape

(24983, 101)

In [113]:
df = df.replace(99, np.nan)

In [114]:
df = df.drop([0], axis = 1)

In [115]:
df.describe()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
count,16452.000000,18184.000000,16642.000000,15608.000000,24977.000000,20431.000000,24976.000000,24981.000000,15752.000000,20324.000000,21821.000000,22423.000000,24981.000000,22515.000000,24977.000000,24975.000000,24980.000000,24979.000000,24978.000000,24980.000000,24921.000000,21433.000000,20033.000000,15879.000000,20816.000000,23750.000000,24904.000000,23990.000000,24934.000000,17977.000000,24656.000000,24948.000000,16704.000000,21563.000000,24952.000000,24965.00000,16881.000000,22858.000000,23080.000000,22398.000000,...,24805.000000,24936.000000,20164.000000,17276.000000,24728.000000,24923.000000,17493.000000,24940.000000,24916.000000,20222.000000,8532.000000,8693.000000,8616.000000,8761.000000,8793.000000,8860.000000,8944.000000,9021.000000,9074.000000,9024.000000,9223.000000,9124.000000,9286.000000,9297.000000,9368.000000,9469.000000,9452.000000,9604.000000,9564.000000,9728.000000,9672.000000,9889.000000,9988.000000,10200.000000,10265.000000,10370.000000,10520.000000,10666.000000,10910.000000,9943.000000
mean,0.904573,0.208460,0.316574,-1.449114,0.403556,1.594447,-0.429662,-0.624085,-0.557031,1.301231,1.805797,1.449290,-1.762882,1.396334,-1.706201,-3.104818,-1.115141,-0.635103,0.154227,-0.933997,2.095626,0.868662,0.109192,-1.687420,0.382373,1.286232,3.189821,1.518830,2.972455,-0.436493,2.198943,3.155988,-1.368648,0.859494,3.006351,3.31250,-1.363987,1.300479,1.067337,0.998635,...,2.459180,2.976021,0.246998,-0.667938,2.273820,2.541538,-0.863868,2.639132,2.559463,0.418405,-0.687494,2.757389,1.082492,-1.553548,-0.270715,2.520047,0.715392,1.751897,0.091667,1.122074,1.836562,0.902718,2.028400,0.694418,0.918927,0.344681,1.900667,2.109940,3.574720,0.612016,2.038527,1.179744,2.504041,1.173482,1.039130,1.528003,1.673750,0.767692,-0.031244,1.354711
std,5.171798,5.597327,5.399806,5.234654,5.255700,4.900540,5.495931,4.929144,5.162224,5.087359,4.916611,4.847879,5.076765,5.122385,5.123290,4.956258,4.398482,4.879306,4.887486,5.100931,4.881075,5.099855,5.110971,5.229222,5.044315,4.856339,4.602144,4.977191,4.792813,4.936876,4.916673,4.456629,5.248554,5.034648,4.597890,4.28162,5.200415,5.011207,4.867559,5.043951,...,4.718655,4.547434,5.143053,5.152378,5.120954,4.701836,5.216555,4.621860,4.662009,4.771693,5.808941,4.529657,5.224221,5.032757,5.427726,4.724249,5.143225,4.932234,5.307042,5.523134,5.338919,5.054666,5.104706,5.118793,5.290301,5.133460,4.782956,4.703183,4.799183,5.259641,5.060445,5.127291,4.611339,5.140307,5.167043,4.852069,5.025076,5.393833,5.176806,5.300409
min,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.95000,-9.950000,-9.950000,-9.950000,-9.950000,...,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000
25%,-2.670000,-4.420000,-4.030000,-5.870000,-3.690000,-1.310000,-5.290000,-4.610000,-4.760000,-2.090000,-1.210000,-1.650000,-6.260000,-2.090000,-6.210000,-7.430000,-4.420000,-4.420000,-3.450000,-5.050000,-0.780000,-2.820000,-3.790000,-6.210000,-3.350000,-1.840000,0.680000,-1.650000,0.340000,-4.170000,-0.630000,0.630000,-5.920000,-2.770000,0.390000,0.97000,-5.870000,-2.140000,-2.140000,-2.520000,...,-0.290000,0.440000,-3.640000,-4.810000,-0.780000,-0.24

In [116]:
df.shape

(24983, 100)

In [117]:
df.head(2)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.4,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.42,NaN,NaN,NaN,-7.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.82,NaN,NaN,NaN,NaN,NaN,-5.63,NaN,NaN,NaN
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.4,-0.87,7.91,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.0,2.77,8.3,7.77,7.33,6.21,7.72,8.98,8.64,8.2,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07


In [119]:
#To find out the average rating of the lowest/highest
joke_mean_sorted = sorted(df.mean())
joke_mean_max = max(joke_mean_sorted)
joke_mean_min = min(joke_mean_sorted)

print('Max ', joke_mean_max, 'Min ', joke_mean_min)
#print('Full list sorted: ')
#joke_mean_sorted

Max  3.6650848950824937 Min  -3.8338796373689923


In [125]:
#Finds specific value of user and joke
df.iloc[13051,17]

-8.5

In [10]:
arr = df.values
arr

array([[100.  ,   4.08,  -0.29, ...,   0.34,  -4.32,   1.07],
       [ 49.  ,    nan,    nan, ...,    nan,    nan,    nan],
       [ 48.  ,    nan,   8.35, ...,    nan,    nan,    nan],
       ...,
       [ 39.  ,    nan,    nan, ...,    nan,    nan,    nan],
       [ 37.  ,    nan,    nan, ...,    nan,    nan,    nan],
       [ 72.  ,   2.43,   2.67, ...,    nan,    nan,    nan]])

In [39]:
def replace(orig, percentage):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """

  new_data = orig.copy()
  rated = np.where(np.isnan(arr))
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [40]:
user_ratings,idx = replace(arr, 0.1)

In [41]:
user_ratings

array([[100.  ,   4.08,  -0.29, ...,   0.34,  -4.32,   1.07],
       [ 49.  ,    nan,    nan, ...,    nan,    nan,    nan],
       [ 48.  ,    nan,   8.35, ...,    nan,    nan,    nan],
       ...,
       [ 39.  ,    nan,    nan, ...,    nan,    nan,    nan],
       [ 37.  ,    nan,    nan, ...,    nan,    nan,    nan],
       [ 72.  ,   2.43,   2.67, ...,    nan,    nan,    nan]])

In [14]:
n_latent_factors = 4

latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [15]:
latent_user_preferences

array([[0.10845279, 0.76831477, 0.28393378, 0.18136221],
       [0.03733479, 0.70630703, 0.28258182, 0.3384407 ],
       [0.40639239, 0.93066475, 0.90244112, 0.81213906],
       ...,
       [0.06086973, 0.34863216, 0.2241462 , 0.08032465],
       [0.06853117, 0.32606904, 0.21267217, 0.42520294],
       [0.4926085 , 0.14443879, 0.55123938, 0.52966231]])

In [16]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    #print(item_id)
    predicted_rating = predict_rating(user_id, item_id)
    err =  predicted_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    mse_history = []
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id, item_id]
                if (rating != 99 and not np.isnan(rating)):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        print('Iteration %d/%d:\tMSE=%.6f' % (iteration, iterations, mse))
        mse_history.append(mse)
    return mse_history

In [18]:
sgd(150)

Iteration 0/150:	MSE=48.447623
Iteration 1/150:	MSE=30.765300
Iteration 2/150:	MSE=26.537907


KeyboardInterrupt: ignored

In [29]:
#prediction for validation set
error_list = []
for i,j in zip(idx[0],idx[1]):
  error_list.append(predict_rating(i,j) - arr[i,j])

final_mse = (np.array(error_list)**2).mean()
print(final_mse)

24.191781598902125


In [30]:
#prediction for nans
np.where(np.isnan(arr))

for i,j in zip(np.where(np.isnan(arr))[0], np.where(np.isnan(arr))[1]):
  arr[i,j] = predict_rating(i,j)

In [31]:
np.where(np.isnan(arr))

(array([], dtype=int64), array([], dtype=int64))

In [20]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[ 8.88359710e+01,  1.00811675e+00,  3.20929824e-01, ...,
         8.34621966e-01, -1.35305732e-01,  1.21169853e+00],
       [ 5.24709099e+01,  6.10803008e-01,  2.43422575e-01, ...,
         5.15250345e-01, -1.34180046e-01,  6.52726817e-01],
       [ 5.86698423e+01,  7.36569201e-01,  2.47614889e-01, ...,
         5.96535418e-01, -1.72896863e-01,  7.84935671e-01],
       ...,
       [ 3.58559814e+01,  4.16165065e-01,  1.26656807e-01, ...,
         3.31029628e-01, -8.62958443e-02,  4.89385755e-01],
       [ 3.69576887e+01,  4.57580360e-01,  1.65579343e-01, ...,
         4.04016059e-01, -3.23554675e-02,  5.01296977e-01],
       [ 7.17827581e+01,  8.77368269e-01,  1.56362211e-01, ...,
         7.27557775e-01,  5.68472376e-02,  1.17715058e+00]])

In [22]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [23]:
comparison_data
# For each data point, the number on the left is the original value from the dataset, the number on the right is the prediction

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(100.0, 88.83597104956029)","(99.0, 1.0081167495042722)","(-0.29, 0.3209298239039419)","(99.0, 0.21455033754572042)","(4.37, -1.6657022037862237)","(-2.38, 0.5378493718519755)","(-9.66, 1.8808494384270582)","(-0.73, -0.508175543276144)","(-5.34, -0.6925240310168188)","(8.88, -0.5018624232379211)","(9.22, 1.6756202202715675)","(6.75, 2.020610150974052)","(8.64, 1.6268444911883926)","(4.42, -1.8086350550832682)","(7.43, 1.669062883945456)","(4.56, -1.7684741733261942)","(-0.97, -3.5125231896207154)","(4.66, -1.1029892278761768)","(-0.68, -0.5435449397662313)","(3.3, 0.27357260970545205)","(-1.21, -0.929504719850192)","(0.87, 2.45531110348409)","(8.64, 1.1477644808110856)","(8.35, 0.12704851233311357)","(9.17, -1.5529004927888896)","(0.05, 0.5539329751481552)","(7.57, 1.6353206678552807)","(4.71, 3.723071267052779)","(0.87, 1.887148095563558)","(-0.39, 3.333510130988839)","(6.99, -0.45819227362440335)","(99.0, 2.8247625984422906)","(-0.92, 3.802497728311728)","(7.14, -1.5271524889127703)","(9.03, 1.0458893717321622)","(-1.8, 3.5170076085803714)","(0.73, 3.944552334473538)","(7.09, -1.34652350427301)","(99.0, 1.5615044488693917)","(-0.87, 1.3310198372706508)",...,"(-6.7, 2.910679442526559)","(-3.35, 3.456081635708679)","(-9.03, 0.38046111569485)","(4.47, -0.5185352234849325)","(4.08, 2.6160844931118787)","(99.0, 3.181898869814657)","(8.74, -0.9384164352287471)","(99.0, 3.268280926026757)","(0.78, 2.9696967161202874)","(7.52, 0.6055791170006471)","(-5.0, -0.6131611908305062)","(2.77, 2.844846946323149)","(8.3, 1.1448303229373613)","(7.77, -1.4650385722270232)","(7.33, -0.14296079271646983)","(6.21, 2.404438291376228)","(7.72, 0.6679784466201384)","(8.98, 1.7487369273329016)","(99.0, 0.3117728765755215)","(8.2, 0.9857942699338148)","(3.93, 1.9448508585581261)","(4.85, 0.8195657949005886)","(4.85, 2.2595541954870852)","(6.07, 0.6481370111240311)","(8.98, 1.1675786797278607)","(4.51, 0.22918401943804578)","(-0.05, 1.6121404169998481)","(3.69, 2.2292149498192897)","(99.0, 3.6641570853881715)","(0.58, 0.6065164170282119)","(2.82, 1.9688682686805237)","(-4.95, 1.1859827676202577)","(-0.29, 2.5650552092087495)","(7.86, 1.172494914521216)","(99.0, 1.154023472889584)","(-2.14, 1.661847199741663)","(3.06, 1.6695539402433444)","(0.34, 0.8346219655702721)","(-4.32, -0.13530573197580142)","(1.07, 1.2116985307186747)"
1,"(49.0, 52.470909916283176)","(nan, 0.6108030079467902)","(nan, 0.24342257542297835)","(nan, 0.04941734173120392)","(nan, -1.0420697797827667)","(99.0, 0.30759983914058253)","(9.27, 1.076055818319254)","(9.03, -0.33823425946253216)","(9.27, -0.38194131156320915)","(nan, -0.3262457418974562)","(nan, 1.032648545226157)","(7.33, 1.1590149488245742)","(7.57, 0.9190505344948322)","(9.37, -1.09831663069818)","(6.17, 0.986516904880886)","(99.0, -1.0713214810777576)","(-6.89, -2.101439606456922)","(99.0, -0.602403191014874)","(9.03, -0.3050725569264415)","(9.03, 0.15208078524043384)","(9.03, -0.5537116359157944)","(99.0, 1.4126426517911952)","(nan, 0.6617293627640983)","(8.25, 0.05216468739292933)","(nan, -0.9156646499016137)","(nan, 0.33966597210949967)","(7.48, 0.9629414173536389)","(7.28, 2.191110341740811)","(7.28, 1.1094381158833295)","(99.0, 1.9487288063611898)","(nan, -0.31182448305257165)","(6.17, 1.7203138834993177)","(7.28, 2.214091562413027)","(nan, -0.9594000904530102)","(nan, 0.6056197869109619)","(99.0, 2.063286076194872)","(7.33, 2.2954914176423906)","(nan, -0.811160031811789)","(6.17, 0.894178213928626)","(9.08, 0.802019348511445)",...,"(6.46, 1.6288160445410897)","(7.28, 1.9603668606043345)","(nan, 0.19246029512752852)","(nan, -0.2378495497520316)","(7.04, 1.4912468939824457)","(7.28, 1.8663651611867103)","(nan, -0.5671597690253425)","(7.28, 1.9258757063660223)","(8.25, 1.71984350245489

In [126]:
#movies dataset

#Matrix for movies and their factors
movie_items_data = pd.read_excel('https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/movies_latent_factors.xlsx', sheet_name='Items')
#Matrix for users and their factors
movie_users_data = pd.read_excel('https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/movies_latent_factors.xlsx', sheet_name='Users')

In [127]:
movie_users_data

,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,4768,-0.204024,0.161079,-0.090447,0.138495,-0.162934,0.163894,0.051502,-0.088582,0.126829,0.065967,0.085008,0.355404,0.007108,-0.118663,-0.039125
1,156,-0.189652,-0.178979,-0.091490,-0.000823,-0.032646,0.177209,-0.098123,-0.068283,-0.011575,0.120866,-0.009931,-0.048606,0.045916,0.113671,0.179873
2,5323,-0.115308,-0.090886,-0.053129,0.018472,-0.068081,-0.004828,0.113005,0.102107,0.034758,0.000693,-0.073712,-0.019460,0.108372,0.054471,-0.109552
3,174,-0.227462,-0.272532,-0.017231,0.054324,0.214755,-0.072639,-0.033122,-0.086508,-0.131479,0.180403,0.095890,-0.082396,0.036767,-0.165438,0.050692
4,4529,-0.014616,-0.102218,-0.107935,0.155784,-0.123362,-0.118228,-0.013549,-0.050622,0.058698,-0.159600,-0.142382,-0.132836,-0.039897,0.129063,0.102669
5,783,-0.020301,-0.031919,-0.036955,0.033690,0.000174,-0.003178,0.054474,0.045424,-0.053308,0.003437,-0.041750,0.059725,-0.021647,0.039873,-0.061857
6,3878,-0.091462,0.215879,-0.180453,0.085408,-0.321094,0.227947,0.053767,0.038017,0.097141,-0.139872,0.029253,0.214467,0.189250,0.065990,0.102776
7,768,0.000819,-0.009229,-0.019228,0.002703,0.012869,0.006655,0.007687,0.018747,0.014705,-0.009256,-0.010116,-0.010051,-0.002108,0.043223,-0.006670
8,4469,-0.030528,-0.011537,-0.042822,-0.014378,0.031338,0.012297,-0.000038,-0.010264,-0.006781,-0.019950,-0.014435,0.010926,0.024400,0.005318,-0.007983
9,1882,-0.083093,-0.029160,0.013748,0.022716,-0.062732,-0.080670,-0.019723,0.013253,-0.091305,-0.090687,0.030638,-0.113364,-0.021325,0.181508,0.022357


In [128]:
#Sorts by specific factor
movie_items_data.sort_values(by='Factor1', ascending=False)

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
86,4327,Charlie's Angels (2000),4.023815,0.882963,1.138274,-0.431058,0.208086,-0.853887,-0.590821,0.416106,0.387390,0.522155,-0.379812,-0.567725,-1.007534,0.794589,-2.247246
37,414,Batman Forever (1995),3.111997,-0.102569,0.514301,-0.720549,0.773486,-0.092841,-1.040723,-0.308222,-1.720265,-0.769622,-0.543568,-0.082656,0.343975,-0.760857,-1.959942
85,3049,Ace Ventura: Pet Detective (1994),2.723720,0.745282,0.029587,-0.328153,0.685947,2.592889,3.457525,-0.780380,-0.219994,-1.313421,0.163827,0.573048,-0.324509,0.358344,0.128814
90,8467,Dumb & Dumber (1994),2.720287,1.217620,-0.101601,-0.119204,1.279939,2.805066,3.524783,-1.283701,-0.133343,-1.343301,-1.009496,1.808762,-1.240696,0.279910,1.616338
68,854,The Mask (1994),2.263142,-0.067761,-0.205463,0.068880,-0.189090,2.078554,2.444922,-0.075971,-0.674767,-0.198560,1.641380,-0.839276,-0.517086,-1.058431,-0.931746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,122,The Lord of the Rings: The Return of the King ...,-2.271110,-2.526300,2.142566,0.341954,2.688624,-0.360610,1.140343,-0.932931,1.356982,1.331742,-0.454221,-0.620533,-0.638745,-0.172093,-0.314937
59,680,Pulp Fiction (1994),-2.382739,2.085395,0.683164,-0.841393,-0.062049,0.326063,0.624981,1.071625,-0.037820,-0.729796,0.412544,-0.033415,0.062671,0.228910,-1.833900
27,238,The Godfather (1972),-2.611462,0.005551,-1.055331,0.150950,0.233674,0.266476,0.605631,0.595702,-0.112474,-1.488997,-0.378031,0.303310,-0.591693,0.400705,-2.267831
33,278,The Shawshank Redemption (1994),-2.780308,-0.088747,-1.821590,-0.157495,0.323285,-0.114768,0.281397,-0.130262,0.028689,-0.810090,-0.229137,0.587575,0.194144,-0.976664,-0.779991


In [129]:
#Selects the whole row of the specified Movie ID
movie_items_data[movie_items_data['Movie ID'] == 13]

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
2,13,Forrest Gump (1994),-2.240888,-0.438815,-2.275177,0.614548,0.904469,0.711919,-0.420876,1.302036,-0.868418,-0.160122,0.633667,0.133138,-0.330276,-2.209004,-0.419092


In [131]:
#Selects the whole row of the specified Movie ID with the minimum loading score (replace .min with .max for maximum)
movie_items_data[movie_items_data['Factor7'] == movie_items_data['Factor7'].min() ]

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
79,1894,Star Wars: Episode II - Attack of the Clones (...,1.480274,-1.711204,1.128254,-1.031576,1.544977,1.662894,-2.001102,0.691944,-3.324583,-0.760322,-0.611091,0.32928,0.627406,0.226807,0.56464


In [133]:
#Calculates the dot product between the User,movie and the factor. Change User/Movie/factor to compare two
np.array(movie_users_data[movie_users_data['User'] == 768].loc[:,'Factor1':]).dot(np.array(movie_items_data[movie_items_data['Movie ID']==1422].loc[:,'Factor1':]).reshape(-1,1))

array([[0.0159661]])

In [31]:
#Selects the whole row of the specified User
movie_users_data[movie_users_data['User'] == 156]

,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
1,156,-0.189652,-0.178979,-0.09149,-0.000823,-0.032646,0.177209,-0.098123,-0.068283,-0.011575,0.120866,-0.009931,-0.048606,0.045916,0.113671,0.179873


In [61]:
np.array(movie_items_data[movie_items_data['Movie ID']==558].loc[:,'Title':])

array([['Spider-Man 2 (2004)', 2.2352233063746585, -0.7881294761636518,
        0.5967221341575716, -0.40008219852561283, -1.246622711078014,
        -1.507203113683306, -0.21991090147438985, -0.4862538356609814,
        0.3316574543781727, -1.8337552665430414, -2.243340340044728,
        -1.1979378607869537, -1.4508122479345416, -2.381119005004064,
        -0.4557528272511082]], dtype=object)

In [32]:
movie_items_data.drop(['Movie ID'],axis=1).values[0][1:]

array([-1.5218476932977845, -1.0385074424076217, 2.0272691992004397,
       0.24793252416549907, -0.5945476708705433, 2.5132599646492184,
       -1.8489100782588614, 0.47670996550851713, -0.22414630395868232,
       -0.7606806322851536, -0.973915043301705, 0.8623788058062587,
       0.4038611510962773, 1.129616009775629, -0.24880587477821164],
      dtype=object)

In [51]:
movie_users_data.shape

(25, 16)

In [49]:
def recommend_movie(user):
  user = np.array(movie_users_data[movie_users_data['User'] == user].loc[:,'Factor1':])
  scores = pd.DataFrame(columns=['ratings','Movie'])
  for i in movie_items_data.drop(['Movie ID'],axis=1).values:
    rating = user.dot(i[1:].reshape(-1,1))
    movie_dict = {'ratings':rating, 'Movie':i[0]}
    scores = scores.append(movie_dict, ignore_index=True)
  return scores.sort_values('ratings', ascending=False).head(3)

In [90]:
def recommends_movie(user, num_recommendations):
  user = np.array(movie_users_data[movie_users_data['User'] == user].loc[:,'Factor1':])
  scores = pd.DataFrame(columns=['ratings','Movie'])
  movie_id = movie_items_data['Movie ID']
  count = 0
  for i in movie_items_data.drop(['Movie ID'],axis=1).values:
    rating = user.dot(i[1:].reshape(-1,1))
    movie_dict = {'ratings':rating,'Movie':i[0],'Movie ID':movie_id[count]}
    scores = scores.append(movie_dict, ignore_index=True)
    count += 1
  return scores.sort_values('ratings', ascending=False).head(num_recommendations)

In [135]:
#Recommends movies based on User. Note: Index is incorrect and needs fixing
#recommender(3947)
recommends_movie(768, 2)

,ratings,Movie,Movie ID
47,[[0.30038024641714217]],Titanic (1997),597.0
13,[[0.09907837413201345]],Pretty Woman (1990),114.0
